from:
https://github.com/martinRenou/ipycanvas

In [1]:
!pip install -U ipycanvas -q

In [2]:
import numpy as np

from ipycanvas import MultiCanvas
from ipywidgets import Image
from ipywidgets import Layout
from time import sleep, time
from ipycanvas import Canvas, hold_canvas
import random

from ipywidgets import Play, IntProgress, HBox, VBox, link
from ipycanvas import Canvas, hold_canvas

In [3]:
# df_maze.py
import random


# Create a maze using the depth-first algorithm described at
# https://scipython.com/blog/making-a-maze/
# Christian Hill, April 2017.

class Cell:
    """A cell in the maze.
    A maze "Cell" is a point in the grid which may be surrounded by walls to
    the north, east, south or west.
    """

    # A wall separates a pair of cells in the N-S or W-E directions.
    wall_pairs = {'N': 'S', 'S': 'N', 'E': 'W', 'W': 'E'}

    def __init__(self, x, y):
        """Initialize the cell at (x,y). At first it is surrounded by walls."""

        self.x, self.y = x, y
        self.walls = {'N': True, 'S': True, 'E': True, 'W': True}

    def has_all_walls(self):
        """Does this cell still have all its walls?"""

        return all(self.walls.values())

    def knock_down_wall(self, other, wall):
        """Knock down the wall between cells self and other."""

        self.walls[wall] = False
        other.walls[Cell.wall_pairs[wall]] = False


class Maze:
    """A Maze, represented as a grid of cells."""

    def __init__(self, nx, ny, ix=0, iy=0):
        """Initialize the maze grid.
        The maze consists of nx x ny cells and will be constructed starting
        at the cell indexed at (ix, iy).
        """

        self.nx, self.ny = nx, ny
        self.ix, self.iy = ix, iy
        self.maze_map = [[Cell(x, y) for y in range(ny)] for x in range(nx)]

    def cell_at(self, x, y):
        """Return the Cell object at (x,y)."""
        return self.maze_map[x][y]
    
    def dimensions(self):
        return self.nx, self.ny

    def __str__(self):
        """Return a (crude) string representation of the maze."""

        maze_rows = ['-' * self.nx * 2]
        for y in range(self.ny):
            maze_row = ['|']
            for x in range(self.nx):
                if self.maze_map[x][y].walls['E']:
                    maze_row.append(' |')
                else:
                    maze_row.append('  ')
            maze_rows.append(''.join(maze_row))
            maze_row = ['|']
            for x in range(self.nx):
                if self.maze_map[x][y].walls['S']:
                    maze_row.append('-+')
                else:
                    maze_row.append(' +')
            maze_rows.append(''.join(maze_row))
        return '\n'.join(maze_rows)

    def write_svg(self, filename):
        """Write an SVG image of the maze to filename."""

        aspect_ratio = self.nx / self.ny
        # Pad the maze all around by this amount.
        padding = 10
        # Height and width of the maze image (excluding padding), in pixels
        height = 500
        width = int(height * aspect_ratio)
        # Scaling factors mapping maze coordinates to image coordinates
        scy, scx = height / self.ny, width / self.nx

        def write_wall(ww_f, ww_x1, ww_y1, ww_x2, ww_y2):
            """Write a single wall to the SVG image file handle f."""

            print('<line x1="{}" y1="{}" x2="{}" y2="{}"/>'
                  .format(ww_x1, ww_y1, ww_x2, ww_y2), file=ww_f)

        # Write the SVG image file for maze
        with open(filename, 'w') as f:
            # SVG preamble and styles.
            print('<?xml version="1.0" encoding="utf-8"?>', file=f)
            print('<svg xmlns="http://www.w3.org/2000/svg"', file=f)
            print('    xmlns:xlink="http://www.w3.org/1999/xlink"', file=f)
            print('    width="{:d}" height="{:d}" viewBox="{} {} {} {}">'
                  .format(width + 2 * padding, height + 2 * padding,
                          -padding, -padding, width + 2 * padding, height + 2 * padding),
                  file=f)
            print('<defs>\n<style type="text/css"><![CDATA[', file=f)
            print('line {', file=f)
            print('    stroke: #000000;\n    stroke-linecap: square;', file=f)
            print('    stroke-width: 5;\n}', file=f)
            print(']]></style>\n</defs>', file=f)
            # Draw the "South" and "East" walls of each cell, if present (these
            # are the "North" and "West" walls of a neighbouring cell in
            # general, of course).
            for x in range(self.nx):
                for y in range(self.ny):
                    if self.cell_at(x, y).walls['S']:
                        x1, y1, x2, y2 = x * scx, (y + 1) * scy, (x + 1) * scx, (y + 1) * scy
                        write_wall(f, x1, y1, x2, y2)
                    if self.cell_at(x, y).walls['E']:
                        x1, y1, x2, y2 = (x + 1) * scx, y * scy, (x + 1) * scx, (y + 1) * scy
                        write_wall(f, x1, y1, x2, y2)
            # Draw the North and West maze border, which won't have been drawn
            # by the procedure above.
            print('<line x1="0" y1="0" x2="{}" y2="0"/>'.format(width), file=f)
            print('<line x1="0" y1="0" x2="0" y2="{}"/>'.format(height), file=f)
            print('</svg>', file=f)

    def find_valid_neighbours(self, cell):
        """Return a list of unvisited neighbours to cell."""

        delta = [('W', (-1, 0)),
                 ('E', (1, 0)),
                 ('S', (0, 1)),
                 ('N', (0, -1))]
        neighbours = []
        for direction, (dx, dy) in delta:
            x2, y2 = cell.x + dx, cell.y + dy
            if (0 <= x2 < self.nx) and (0 <= y2 < self.ny):
                neighbour = self.cell_at(x2, y2)
                if neighbour.has_all_walls():
                    neighbours.append((direction, neighbour))
        return neighbours

    def make_maze(self):
        # Total number of cells.
        n = self.nx * self.ny
        cell_stack = []
        current_cell = self.cell_at(self.ix, self.iy)
        # Total number of visited cells during maze construction.
        nv = 1

        while nv < n:
            neighbours = self.find_valid_neighbours(current_cell)

            if not neighbours:
                # We've reached a dead end: backtrack.
                current_cell = cell_stack.pop()
                continue

            # Choose a random neighbouring cell and move to it.
            direction, next_cell = random.choice(neighbours)
            current_cell.knock_down_wall(next_cell, direction)
            cell_stack.append(current_cell)
            current_cell = next_cell
            nv += 1
            
    def write_to_canvas(self, canvas, maze_height, maze_padding ): 
        ' draw the maze onto the canvas '

        aspect_ratio = self.nx / self.ny
        # Pad the maze all around by this amount.
        padding = maze_padding
        
        # Height and width of the maze image (excluding padding), in pixels
        height = maze_height
        width = int(height * aspect_ratio)
        
        # Scaling factors mapping maze coordinates to image coordinates
        scy, scx = height / self.ny, width / self.nx

        canvas.line_width = 4
        canvas.line_cap = 'square'    
        
        canvas.stroke_style = '#000'        
        canvas.set_line_dash([0,0])        

        def draw_wall( x1,y1,x2,y2 ):
            canvas.begin_path()
            canvas.move_to(x1 + padding, y1 + padding)
            canvas.line_to(x2 + padding, y2 + padding)
            canvas.stroke()

        # Draw the "South" and "East" walls of each cell, if present (these
        # are the "North" and "West" walls of a neighbouring cell in
        # general, of course).
        for x in range(self.nx):
            for y in range(self.ny):
                if self.cell_at(x, y).walls['S']:
                    x1, y1, x2, y2 = x * scx, (y + 1) * scy, (x + 1) * scx, (y + 1) * scy
                    draw_wall(x1, y1, x2, y2)
                if self.cell_at(x, y).walls['E']:
                    x1, y1, x2, y2 = (x + 1) * scx, y * scy, (x + 1) * scx, (y + 1) * scy
                    draw_wall(x1, y1, x2, y2)

        # Draw the North and West maze border, which won't have been drawn
        # by the procedure above.
        draw_wall(0, 0, 0, height)        
        draw_wall(0, 0, width, 0)                    

In [7]:
sprites = Image.from_file('../Images/BabyRobot64_Sprites.png')
canvas = Canvas(width=132, height=328, sync_image_data=True)   

def save_to_file(*args, **kwargs):
    canvas.to_file('Images/my_test_file.png')

# Listen to changes on the ``image_data`` trait and call ``save_to_file`` when it changes.
canvas.observe(save_to_file, 'image_data')

canvas.draw_image( sprites, 0, 0 )

In [8]:
canvas

Canvas(height=328, image_data=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x84\x00\x00\x01H\x08\x06\x00\x…

In [9]:
# wait for image to load
sleep(1.0)

image_data = canvas.get_image_data(x=0, y=0, width=64, height=64)

canvas2 = Canvas(width=64, height=64)   
canvas2.put_image_data( image_data, 0, 0 )
canvas2

Canvas(height=64, width=64)

In [10]:
image_data = canvas.get_image_data(x=64, y=0, width=64, height=64)
canvas3 = Canvas(width=64, height=64)   
canvas3.put_image_data( image_data, 0, 0 )
canvas3

Canvas(height=64, width=64)

In [11]:
def draw_square(canvas, pos, width, color):
    canvas.fill_style = color
    canvas.fill_rect(pos, pos, width, width)
    
def draw_rect(canvas, width, height, color):
    canvas.fill_style = color
    canvas.fill_rect(0, 0, width, height)    

In [12]:
m = MultiCanvas(n_canvases=3, width=256, height=256, sync_image_data=True)

In [13]:
# draw the base of the grid
draw_square(m[0], 0, 256, 'orange')

In [14]:
# draw the grid onto the canvas
with hold_canvas(m[1]):
    for y in range(4):   
        for x in range(4):   
            m[1].stroke_rect(64 * x, 64 * y, 64, 64)

In [15]:
import pdb

from enum import Enum

class Direction(Enum):
    North = 1
    East = 2
    South = 3
    West = 4
    
    # get the enum name without the class
    def __str__(self): return self.name


class RobotPosition():
    
    sprite_count = 0
    move_count = 0   
    canvas_sprites = []

    
    def __init__(self, canvas, x_size = 256, y_size = 256, initial_sprite = 4, maze = None):
        
        self.canvas = canvas
        self.maze = maze
                        
        self.x = 0
        self.y = 0     
        self.step = 4
        self.robot_size = 64
        
        # the number of steps before a sprite change
        self.sprite_change = 4
        
        if maze is None:
            self.x_size = x_size
            self.y_size = y_size
        else:
            x,y = maze.dimensions()
            self.x_size = x * self.robot_size
            self.y_size = y * self.robot_size
                       
        self.sprite_index = initial_sprite
        self.load_sprites() 
        
    def get_number_of_sprites(self):
        return len(self.canvas_sprites)
    
    def get_cell_position(self):
        return self.x//self.robot_size, self.y//self.robot_size
    
    def set_cell_position(self, grid_x, grid_y):
        self.x = grid_x * self.robot_size
        self.y = grid_y * self.robot_size    
        
    def get_array(self,*args, **kwargs):                
        ' callback to split the sprite sheet into individual sprites '
        index = 0
        for row in range(5):
            for col in range(2):                                 
                x = col * (self.robot_size + 1)                    
                y = row * (self.robot_size + 1)
        
                # copy the sprite from the sprite sheet
                image_data = self.sprite_canvas.get_image_data( x, y, self.robot_size)

                # put the sprite onto its own canvas
                canvas = Canvas(width=self.robot_size, height=self.robot_size)
                canvas.put_image_data( image_data, 0, 0 )  
                self.canvas_sprites.append( canvas )
        
        # add a sprite to the display
        self.canvas.clear()
        self.draw()         
        
    def load_sprites(self):        
        ' load the sprite sheet and when loaded callback to split it into individual sprites '
        sprites = Image.from_file('../Images/BabyRobot64_Sprites.png')
        self.sprite_canvas = Canvas(width=132, height=328, sync_image_data=True)   
        self.sprite_canvas.draw_image( sprites, 0, 0 )                 
        self.sprite_canvas.observe(self.get_array, 'image_data')                
            
    def update_sprite(self):
        ' randomly change to the next sprite '        
        self.sprite_count += 1
        if self.sprite_count > self.sprite_change: 
            self.sprite_count = 0   
            self.sprite_index = self.sprite_index + random.randint(-1,+1)   
            if self.sprite_index < 0: 
                self.sprite_index = 0
            if self.sprite_index >= self.get_number_of_sprites():
                self.sprite_index = (self.get_number_of_sprites()-1)    
                              
    def draw_sprite(self,index):   
        ' remove the last sprite and add the new one at the current position '
        with hold_canvas(self.canvas):
            self.canvas.clear_rect(self.x, self.y, self.robot_size)                        
            self.canvas.draw_image(self.canvas_sprites[index], self.x, self.y )                       
        
    def draw(self):    
        ' add the current sprite at the current position '        
        self.draw_sprite(self.sprite_index)
        self.update_sprite()               
            
    def move(self,direction):        
        ' move from one square to the next in the specified direction '
        
        cell = None
        if self.maze is not None:
            x, y = self.get_cell_position()
            cell = self.maze.cell_at( x, y )            
            if direction == Direction.North and cell.walls['N']: return
            if direction == Direction.South and cell.walls['S']: return
            if direction == Direction.East and cell.walls['E']: return
            if direction == Direction.West and cell.walls['W']: return
        
        move_method_name = f"move_{direction}"        
        for step in range(64//self.step):
            getattr(self,move_method_name)()  
            self.draw()  
            sleep(0.10) # pause between each move step        
            
        self.move_count += 1
        
    def move_East(self):        
        if self.x < (self.x_size - self.robot_size):
            self.x += self.step
            
    def move_West(self):
        if self.x > 0: 
            self.x -= self.step            
            
    def move_North(self):
        if self.y > 0:           
            self.y -= self.step
            
    def move_South(self):
        if self.y < (self.y_size - self.robot_size):
            self.y += self.step

        

In [16]:
m

MultiCanvas(height=256, sync_image_data=True, width=256)

In [17]:
robot_position = RobotPosition(m[2])

In [18]:
# for i in range(20): 
#     robot_position.move( random.choice(list(Direction)))      

In [19]:
def save_to_file(*args, **kwargs):    
    global robot_position
    m.to_file(f'Images/test_{robot_position.move_count}.png')   
        
m.observe(save_to_file, 'image_data')                         

In [20]:
play = Play(interval=500, min=1, max=20, step=1)
progress = IntProgress(min=1, max=20, step=1)

link((play, 'value'), (progress, 'value'))
robot_position = RobotPosition(m[2])

def on_update(*args):   
    with hold_canvas(m):
        robot_position.move( random.choice(list(Direction)))  

play.observe(on_update, 'value')

layout = Layout(width='256px', height='296px')
VBox((m, HBox((play, progress))),layout=layout)

In [21]:
sprite1 = Image.from_file('../Images/BabyRobot64_GL.png')

def move_one_square( canvas ):
    for x in range(0,64,4):
        with hold_canvas(canvas):
            canvas.clear()
            canvas.draw_image(sprite1, x, 64)   
        sleep(0.10)

In [22]:
m[2].clear()
move_one_square(m[2]) 

In [23]:
def save_to_file(*args, **kwargs):
    print(args)
    m.to_file(f"Images/my_file_.png")         

In [25]:
# canvas = Canvas(width=200, height=200, sync_image_data=True)

# Perform some drawings...

m.sync_image_data = True
m.to_file('Images/my_file.png')
m.sync_image_data = False

In [26]:
from PIL import Image as PILImage
import numpy as np

sprite3 = np.array(PILImage.open('../Images/smoke_texture0.png'))

In [27]:
m[2].put_image_data(sprite3, 50, 150)

# Maze Test

In [28]:
maze_rows = 6
maze_cols = 10
cell_pixels = 64
padding = 2

maze_width = (maze_cols * cell_pixels)
maze_height = (maze_rows * cell_pixels)

m = MultiCanvas(n_canvases=3, 
                width= maze_width + (padding*2), 
                height= maze_height + (padding*2), 
                sync_image_data=True)

In [29]:
# from df_maze import Maze

# Maze dimensions (ncols, nrows)
nx, ny = 15, 15
# Maze entry position
ix, iy = 0, 0

maze = Maze(maze_cols, maze_rows, ix, iy)
maze.make_maze()

# print(maze)
maze.write_svg('maze.svg')

In [30]:
# draw the base of the grid
max_x = maze_width + (padding*2)
max_y = maze_height + (padding*2)
draw_rect(m[0], max_x, max_y, 'orange')

# add dashed lines showing grid
m[1].clear()
m[1].stroke_style = '#777'
m[1].line_width = 1
m[1].set_line_dash([4,8])

with hold_canvas(m[1]):
    # draw the grid onto the canvas
    for y in range(maze_rows):   
        for x in range(maze_cols):   
            m[1].stroke_rect(64 * x + padding, 64 * y + padding, 64, 64)
            
    # add the maze
    maze.write_to_canvas(m[1],maze_height,padding)                       

    # add the start
    m[0].fill_style = '#d00'
    m[0].fill_rect(padding, padding, 64, 64)         
    m[1].fill_style = '#fff'
    m[1].text_align = 'left'
    m[1].font = 'bold 16px sans-serif'
    m[1].fill_text(str("START"), padding + 6, padding + 37)    
    
    # add the exit
    m[0].fill_style = 'green'
    m[0].fill_rect(max_x-66, max_y-66, 64, 64)    
    m[1].fill_style = '#fff'
    m[1].text_align = 'left'
    m[1].font = 'bold 20px sans-serif'
    m[1].fill_text(str("EXIT"), max_x - 56, max_y -26)

In [31]:
m

MultiCanvas(height=388, sync_image_data=True, width=644)

In [32]:
robot_position = RobotPosition(m[2],maze=maze)

In [36]:
# for i in range(60): 
#     robot_position.move( random.choice(list(Direction))) 

In [37]:
# add a callback function to the canvas to save each step to a PNG
def save_to_file(*args, **kwargs):    
    global robot_position
    m.to_file(f'Images/test_{robot_position.move_count}.png')   
        
m.observe(save_to_file, 'image_data') 

In [38]:
play = Play(interval=500, min=1, max=40, step=1)
progress = IntProgress(min=1, max=40, step=1)

link((play, 'value'), (progress, 'value'))
robot_position = RobotPosition(m[2],maze=maze)

def on_update(*args):   
    robot_position.move( random.choice(list(Direction)))  

play.observe(on_update, 'value')

pixel_width = maze_cols * cell_pixels
pixel_height = maze_rows * cell_pixels

layout = Layout(width=f'{pixel_width}px', height='{pixel_height}px')
VBox((m, HBox((play, progress))),layout=layout)

## Initial Level

In [236]:
maze_rows = 3
maze_cols = 5
cell_pixels = 64
padding = 2

maze_width = (maze_cols * cell_pixels)
maze_height = (maze_rows * cell_pixels)

m = MultiCanvas(n_canvases=3, 
                width= maze_width + (padding*2) + 100, 
                height= maze_height + (padding*2), 
                sync_image_data=True)

m.observe(save_to_file, 'image_data')

In [237]:
max_x = maze_width + (padding*2)
max_y = maze_height + (padding*2)

In [238]:
def grid_to_pixels( grid_pos, xoff = 0, yoff = 0 ):
    x = (grid_pos[0] * cell_pixels) + padding + xoff
    y = max_y - ((grid_pos[1] + 1) * cell_pixels) - padding + yoff   
    return x,y

In [239]:
# draw the base of the grid
draw_rect(m[0], max_x, max_y, 'orange')

start = [0,1]
end = [3,0]

with hold_canvas(m[1]):
    
    # add dashed lines showing grid
    m[1].clear()
    m[1].stroke_style = '#777'
    m[1].line_width = 1
    m[1].set_line_dash([4,8])    
    
    # draw the grid onto the canvas
    for y in range(maze_rows):   
        for x in range(maze_cols):   
            m[1].stroke_rect(64 * x + padding, 64 * y + padding, 64, 64)
                              

    # add the start
    start_x, start_y = grid_to_pixels( start )
    m[0].fill_style = '#ed1818'
    m[0].fill_rect(start_x, start_y, 64, 64)         
    m[1].fill_style = '#fff'
    m[1].text_align = 'left'
    m[1].font = 'bold 16px sans-serif'
#     m[1].fill_text(str("START"), start_x + 6, start_y + 37)    
    
    # add the exit
    end_x, end_y = grid_to_pixels( end )
    m[0].fill_style = 'green'
    m[0].fill_rect(end_x, end_y, 64, 64)    
    m[1].fill_style = '#fff'
    m[1].text_align = 'left'
    m[1].font = 'bold 20px sans-serif'
    m[1].fill_text(str("EXIT"), end_x + 10, end_y + 40) 
    
    # draw the border
    m[1].stroke_style = '#000'
    m[1].line_width = 5
    m[1].set_line_dash([0,0])
    m[1].stroke_rect(2, 2, max_x-4, max_y-4)
    
    
    x1,y1 = grid_to_pixels( [1,1], 2, 2 )
    x2,y2 = grid_to_pixels( [4,0], -2, -2 )      
    m[1].fill_style = '#fff'
    m[1].fill_rect(x1,y1,x2-x1,y2-y1)
    m[1].stroke_rect(x1,y1,x2-x1,y2-y1)      
    
    
    # draw the compass       
    m[1].line_width = 2
    m[1].fill_style = '#000'
    m[1].stroke_style = '#000'
    m[1].begin_path()
    m[1].move_to(max_x + 57, 25)
    m[1].line_to(max_x + 57, 65)      
    m[1].move_to(max_x + 37, 45)
    m[1].line_to(max_x + 77, 45)      
    m[1].stroke()    
    
    m[1].font = 'bold 20px sans-serif'
    m[1].fill_text(str("N"), max_x + 49, 18)     
    m[1].fill_text(str("E"), max_x + 82, 52)     
    m[1].fill_text(str("S"), max_x + 51, 85)     
    m[1].fill_text(str("W"), max_x + 15, 52)      

In [245]:
# add value to each grid cell
def add_value( canvas, pos, value ):
    end_x, end_y = grid_to_pixels( pos )   
    
    canvas.clear_rect(end_x+21,end_y+24,19,18)  
#     canvas.fill_style = 'orange'
#     canvas.fill_rect(end_x+21,end_y+24,20,18)
    canvas.fill_style = '#002'
    canvas.text_align = 'left'
    canvas.font = 'bold 20px sans-serif'
    canvas.fill_text(f"{value}", end_x + 22, end_y + 40)  
    
def add_robot_value( canvas, pos, value ):
    end_x, end_y = grid_to_pixels( pos )   
  
    # clear the background number 
    m[1].clear_rect(end_x+21,end_y+24,20,20)  
    
    # add a number on top of baby robot
    canvas.fill_style = '#ccc'
    canvas.fill_rect(end_x+21,end_y+27,20,18)
    canvas.fill_style = '#002'
    canvas.text_align = 'left'
    canvas.font = 'bold 20px sans-serif'
    canvas.fill_text(f"{value}", end_x + 22, end_y + 43)     

In [246]:
m

MultiCanvas(height=196, image_data=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xa8\x00\x00\x00\xc4\x08\x…

In [247]:
robot_position = RobotPosition(m[2])
robot_position.set_cell_position(0,1)

In [248]:
play = Play(interval=500, min=1, max=40, step=1)
progress = IntProgress(min=1, max=40, step=1)

link((play, 'value'), (progress, 'value'))

robot_position = RobotPosition(m[2])
robot_position.set_cell_position(0,1)
robot_position.sprite_change = 1    

def on_update(*args):   
    robot_position.draw()
    add_robot_value( m[2], [0,1], -4 )

play.observe(on_update, 'value')

pixel_width = maze_cols * cell_pixels
pixel_height = maze_rows * cell_pixels

layout = Layout(width=f'{m.width}px', height='{m.height}px')
VBox((m, HBox((play, progress))),layout=layout)

In [249]:
from math import pi
  
text_canvas = m[1]    
    
pos = [3,0]
for x in range( 1, 4 ):
    pos[0] = 3 - x 
    add_value( text_canvas, pos, -x )     
    
pos = [0,0]    
for y in range( 4, 7 ):
    pos[1] = y - 3 
    add_value( text_canvas, pos, -y )     
    
pos = [4,0]    
for y in range( 1, 4 ):
    pos[1] = y - 1 
    add_value( text_canvas, pos, -y ) 
    
for x in range( 4, 7 ):
    pos[0] = 7 - x 
    add_value( text_canvas, pos, -x )    
    print(x,pos)


4 [3, 2]
5 [2, 2]
6 [1, 2]
